In [1]:
import branca
import pandas as pd
import geopandas as gpd

from siuba import *

from shared_utils import map_utils
from shared_utils import calitp_color_palette as cp
from utils import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
coverage_gdf = gpd.read_parquet(f'{GCS_FILE_PATH}block_accessibility_metrics.parquet')

In [3]:
coverage_gdf.head(3)

,county,tract,block,geo_id,geometry,block_pop,area,calitp_itp_id,rt_accessible,rt,accessible,static,rt_milestone
0,65,42628,4008,60650426284008,"POLYGON ((257831.965 -463783.472, 258008.040 -...",103,15986.958897,269,False,True,False,True,False
1,65,42711,3025,60650427113025,"POLYGON ((259577.302 -476173.973, 259651.066 -...",42,29609.218158,269,False,True,False,True,False
2,65,43401,1023,60650434011023,"POLYGON ((281412.631 -469351.296, 281613.718 -...",21,19971.850439,13,False,False,False,True,True


In [4]:
coverage_gdf.geometry = coverage_gdf.buffer(0).simplify(tolerance=10)

In [5]:
def subset_and_dissolve(gdf, metric):
    df = ( (gdf.copy() 
         >> filter(_[metric])
         >> select(_.county, _.calitp_itp_id, 
                   _.static, _.accessible, _.rt,
                   _.rt_accessible, _.rt_milestone, 
                   _.geometry) )
        .dissolve(by='calitp_itp_id')
        .reset_index()
    )
    
    df['geometry'] = df.geometry.buffer(150).simplify(tolerance=75)
    
    return df

In [6]:
df_rt_acc = subset_and_dissolve(coverage_gdf, "rt_accessible")
df_rt = subset_and_dissolve(coverage_gdf, "rt")
df_rt_milestone = subset_and_dissolve(coverage_gdf, "rt_milestone")
df_acc = subset_and_dissolve(coverage_gdf, "accessible")
df_static = subset_and_dissolve(coverage_gdf, "static")

In [7]:
df_rt_milestone.head()

,calitp_itp_id,geometry,county,static,accessible,rt,rt_accessible,rt_milestone
0,6,"MULTIPOLYGON (((159150.448 -441360.404, 158988...",1,True,False,False,False,True
1,13,"MULTIPOLYGON (((503362.363 -572298.673, 503322...",65,True,False,False,False,True
2,14,"MULTIPOLYGON (((197009.198 -460703.159, 197546...",59,True,False,False,False,True
3,17,"POLYGON ((178367.645 -429595.729, 178330.835 -...",37,True,False,False,False,True
4,33,"POLYGON ((-200140.715 -14724.558, -200234.764 ...",1,True,False,False,False,True


In [8]:
popup_dict = {"calitp_itp_id": 'ITP ID',
            "rt_accessible": "Accessible Trip and Stop, RT Feed Available",
            "rt": "RT Feed Available",
            "rt_milestone": "RT Feed in MSD Milestones",
            "accessible": "Accessible Trip and Stop",
            "static": "GTFS Static Present",
             }

In [9]:
LAYERS_DICT = {
    "RT Accessible": {"df": df_rt_acc,
        "plot_col": "rt_accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[:1],
    },
    "RT": {"df": df_rt,
        "plot_col": "rt",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[4:5],
    },
    "RT Milestones": {"df": df_rt_milestone,
        "plot_col": "rt_milestone",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2],
    },
    "Accessible": {"df": df_acc,
        "plot_col": "accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3],
    },
    "GTFS Static": {"df": df_static,
        "plot_col": "static",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4],
    }
}

In [10]:
# map_utils.make_folium_multiple_layers_map?

rt_accessible, cp.CALITP_CATEGORY_BRIGHT_COLORS[:1]
rt, cp.CALITP_CATEGORY_BRIGHT_COLORS[4:5]
rt_milestone, cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2]
accessible, cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3]
static, cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4]

In [25]:
# Make legend and save as static image
colors = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[:1][0], 
    cp.CALITP_CATEGORY_BRIGHT_COLORS[4:5][0],
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2][0],
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3][0],
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4][0]
]

colorscale = branca.colormap.StepColormap(
                colors=colors,
)

print("RT Accessible      RT       RT Milestone   Accessible      Static")
display(colorscale)

RT Accessible      RT       RT Milestone   Accessible      Static


In [11]:
fig = map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 800, fig_height = 800, 
        zoom=9, 
        centroid = [36.5, -120], 
        title=f"Statewide Feed Coverage",
        legend_dict = {"legend_url": "", "legend_bottom": 85, "legend_left": 5}
    )

In [13]:
#fig

In [ ]:
#fig.save('statewide_coverage_map.html')